In [2]:
import sys
sys.path.append('..')

from utils.common import read_beacon_coords, read_beacon_data, process_beacon_data, process_beacon_data_for_pos
import argparse
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import pdb

## verify the loc algo correctness with dummy point

In [22]:
from scipy.optimize import minimize
import argparse


args = argparse.Namespace(beacon_coords_file='../data0829ad/beacon_coords.txt', beacon_data_dir='../data0829ad/', beacon_data_file='../data0829ad/bluetoothData(84).txt', update_frequency=10)


def trilaterate(beacons):
    def error(point, beacons):
        return sum((np.linalg.norm(np.array(point) - np.array(b[:3])) - b[3])**2 for b in beacons)
    initial_guess = np.mean(beacons, axis=0)[:3]
    bounds = [(None, None), (None, None), (0.9, 1.1)]
    result = minimize(error, initial_guess, args=(beacons,), method='L-BFGS-B', bounds=bounds)
    # result = minimize(error, initial_guess, args=(beacons,), method=self.config['trilateration']['method'])
    return result.x


coords = [[3.0, y, 1.0] for y in np.arange(0, 6, 0.2)]
beacon_coords = [[0, 0, 0], [6, 3, 0], [0, 6, 0]]

def solve_dis(coord):
    return [np.linalg.norm(np.array(coord) - np.array(bc)) for bc in beacon_coords]

distance_input = [solve_dis(coord) for coord in coords]
beacon_inputs = []

for i, d in enumerate(distance_input):
    temp = []
    for j in range(len(beacon_coords)):
        new_beacon_coord = beacon_coords[j][:]  # 复制当前的 beacon 坐标
        new_beacon_coord.append(d[j])  # 添加距离值
        temp.append(new_beacon_coord)
    beacon_inputs.append(temp)

print(beacon_inputs[0])
print(distance_input[0])
error = 0
for i in range(len(coords)):
    result = trilaterate(beacon_inputs[i])
    error += np.linalg.norm(np.array(result) - np.array(coords[i]))
print(error)

[[0, 0, 0, 3.1622776601683795], [6, 3, 0, 4.358898943540674], [0, 6, 0, 6.782329983125268]]
[3.1622776601683795, 4.358898943540674, 6.782329983125268]
0.00011799917933213029
